메모

In [1]:
map (*2) [1,2,3,4,5]

[2,4,6,8,10]

In [2]:
map :: (a -> b) -> [a] -> [b]
map f []     = []
map f (x:xs) = f x : map f xs

<span class='hoogle-name'><a target='_blank' href='http://hackage.haskell.org/packages/archive/base/latest/doc/html/Prelude.html#v:concat'>concat</a> &#x2237; [[a]] &#x2192; [a]</span>
<span class='hoogle-sub'>(<span class='hoogle-head'>package</span>
<span class='hoogle-package'>base</span>, <span class='hoogle-head'>module</span> <span class='hoogle-module'>Prelude</span>)</span>
<div class='hoogle-doc'>
<div class='hoogle-text'>Concatenate a list of lists. e.g.,</div>

```haskell
concat [xs1, xs2, xs3] == xs1 ++ xs2 ++ xs3
concat xss == foldr (++) [] xss
```

</div>

In [3]:
concat [[1,2], [3,4,5], [6,7,8,9]]

[1,2,3,4,5,6,7,8,9]

In [4]:
foldr (++) [] [[1,2], [3,4,5], [6,7,8,9]]

Line 1: Use concat
Found:
foldr (++) []
Why not:
concat

[1,2,3,4,5,6,7,8,9]

In [5]:
:doc foldr

In [6]:
:doc map

In [7]:
-- 변수 이름 타입은 문자열로 한다
type Nm = String

-- 람다식 문법 구조
data Tm = Var Nm | Lam Nm Tm | App Tm Tm  deriving Show

-- 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x t) = "\\" ++ x ++ " -> " ++ ppTm t
ppTm (App t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam{}) = paren (ppTm t)
  ppt t         = ppTm t
  pps s@(Var{}) = ppTm s
  pps s         = paren (ppTm s)

paren s = "(" ++ s ++ ")"

In [8]:
idTm = Lam "x" (Var "x")
ttTm = Lam "x" (Lam "y" (Var "x")) 
ffTm = Lam "x" (Lam "y" (Var "y")) 

putStr . ppTm $ Var "x"
putStr . ppTm $ idTm
putStr . ppTm $ App idTm idTm
putStr . ppTm $ App (App ttTm (Var "u")) (Var "w")

x

\x -> x

(\x -> x) (\x -> x)

(\x -> \y -> x) u w

In [9]:
-- 하스켈에서 그대로 붙여넣으면 실행도 어느 정도 가능하게
let z = "z" in (\x -> x) ((\x -> x) z)

Line 1: Use id
Found:
\ x -> x
Why not:
idLine 1: Use id
Found:
\ x -> x
Why not:
id

"z"

In [91]:
:ext DeriveFunctor
-- Tm의 구조에서 딱 한군데를 지정해 가리키기 위한 구조
data TmF tm = Here tm -- 여기야 여기
            | Var' Nm | Lam' Nm (TmF tm) | App' (TmF tm) (TmF tm)
  deriving (Show, Functor)

type Tm' = TmF Tm

{-
-- Functor 클래스를 빼고 생각하면 대략 다음과 같다고 보면 된다

data Tm' = Here Tm -- 여기야 여기
         | Var' Nm | Lam' Nm Tm' | App' Tm' Tm'
  deriving Show

-}


-- 보기좋게 문자열로 변환해주는 함수
ppTm' (Here t) = "[" ++ ppTm t ++ "]"
ppTm' (Var' x) = x
ppTm' (Lam' x t) = "\\" ++ x ++ " -> " ++ ppTm' t
ppTm' (App' t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam'{})      = paren (ppTm' t)
  ppt t@(Here(Lam{})) = paren (ppTm' t)
  ppt t               = ppTm' t
  pps s@(Var'{})      = ppTm' s
  pps s@(Here(Var{})) = ppTm' s
  pps s               = paren (ppTm' s)

-- Tm을 Tm'으로 변환
tm2tm' (Var x) = Var' x
tm2tm' (Lam x t) = Lam' x (tm2tm' t)
tm2tm' (App t s) = App' (tm2tm' t) (tm2tm' s)

-- Tm'을 으로 변환 (Here 정보는 사라진다)
tm'2tm (Here t) = t
tm'2tm (Var' x) = Var x
tm'2tm (Lam' x t) = Lam x (tm'2tm t)
tm'2tm (App' t s) = App (tm'2tm t) (tm'2tm s) 

In [92]:
idTm' = tm2tm' idTm

Here(App idTm idTm)
putStr . ppTm' $ Here(App idTm idTm)

App' idTm' (Here(App idTm idTm))
putStr . ppTm' $ App' idTm' (Here(App idTm idTm))

Here (App (Lam "x" (Var "x")) (Lam "x" (Var "x")))

[(\x -> x) (\x -> x)]

App' (Lam' "x" (Var' "x")) (Here (App (Lam "x" (Var "x")) (Lam "x" (Var "x"))))

(\x -> x) ([(\x -> x) (\x -> x)])

In [93]:
-- 아래 람다식의 beta redex를 모두 찾아 각각의 경우를 하나씩 Here로 표시해 보자
putStr . ppTm $ Lam "y" (App idTm (App idTm (Var "y")))

\y -> (\x -> x) ((\x -> x) y)

In [94]:
-- 다음의 두 가지 경우가 있다
putStr . ppTm' $ Lam' "y" (Here(App idTm (App idTm (Var "y"))))
putStr . ppTm' $ Lam' "y" (App' idTm' (Here(App idTm (Var "y"))))

\y -> [(\x -> x) ((\x -> x) y)]

\y -> (\x -> x) ([(\x -> x) y])

In [95]:
-- 아래 람다식에는 beta redex가 3개 있는데 모두 찾아 각각을 Here로 표시해 보라
putStr . ppTm $ App idTm (App idTm idTm `App` App idTm idTm)

(\x -> x) ((\x -> x) (\x -> x) ((\x -> x) (\x -> x)))

In [96]:
-- 여기에 해보세요

In [97]:
-- beta redex를 모두 찾아 리스트로 나열하는 함수를 작성하라
-- beta redex가 없을 경우는 빈 리스트가 결과값이 된다
redexes :: Tm -> [Tm']
redexes (Var x) = []
redexes (Lam x t) = [Lam' x t' | t' <- redexes t]
redexes (App t s) = here
                 ++ [App' t' (tm2tm' s) | t' <- redexes t]
                 ++ [App' (tm2tm' t) s' | s' <- redexes s]
  where
  here = case t of Lam{} -> [Here (App t s)]
                   _     -> []

In [98]:
import IHaskell.Display

dispTm t = Display [html("<code>"++dpTm t++"</code>")]
dispTm' t = Display [html("<code>"++dpTm' t++"</code>")]

dpTm (Var x) = x
dpTm (Lam x t) = "λ" ++ x ++ "." ++ dpTm t
dpTm (App t s) = dpt t ++ " " ++ dps s
  where
  dpt t@(Lam{}) = paren (dpTm t)
  dpt t         = dpTm t
  dps s@(Var{}) = dpTm s
  dps s         = paren (dpTm s)
  
dpTm' (Here t) = "<span style='color:red'>" ++ dpTm t ++ "</span>"
dpTm' (Var' x) = x
dpTm' (Lam' x t) = "λ" ++ x ++ "." ++ dpTm' t
dpTm' (App' t s) = dpt t ++ " " ++ dps s
  where
  dpt t@(Lam'{})      = paren (dpTm' t)
  dpt t@(Here(Lam{})) = paren (dpTm' t)
  dpt t               = dpTm' t
  dps s@(Var'{})      = dpTm' s
  dps s@(Here(Var{})) = dpTm' s
  dps s               = paren (dpTm' s)

dispTm $ Lam "y" (App idTm (App idTm (Var "y")))

dispTm' $ Lam' "y" (Here(App idTm (App idTm (Var "y"))))

dispTm' $ Lam' "y" (App' idTm' (Here(App idTm (Var "y"))))

λy.(λx.x) ((λx.x) y)

λy. (λx.x) ((λx.x) y)

λy.(λx.x) ( (λx.x) y )

In [99]:
redexes $ Lam "y" (App idTm (App idTm (Var "y")))

mapM_ (putStrLn . ppTm') . redexes $ Lam "y" (App idTm (App idTm (Var "y")))

map dispTm' . redexes  $ Lam "y" (App idTm (App idTm (Var "y")))

[Lam' "y" (Here (App (Lam "x" (Var "x")) (App (Lam "x" (Var "x")) (Var "y")))),Lam' "y" (App' (Lam' "x" (Var' "x")) (Here (App (Lam "x" (Var "x")) (Var "y"))))]

\y -> [(\x -> x) ((\x -> x) y)]
\y -> (\x -> x) ([(\x -> x) y])

λy. (λx.x) ((λx.x) y)

λy.(λx.x) ( (λx.x) y )

## freshness, alpha-equivalence, capture avoiding substitution
beta reduction을 구현하기 위한 갈길이 멀도다

In [100]:
import Data.List ( (\\), union )

fv (Var x) = [x]
fv (Lam x t) = fv t \\ [x]
fv (App t s) = fv t `union` fv s

In [101]:
dispTm (Lam "x" (App (Var "x") (Var "y")))
fv (Lam "x" (App (Var "x") (Var "y")))

dispTm (App (Lam "x" (App (Var "x") (Var "y"))) (Var "z"))
fv (App (Lam "x" (App (Var "x") (Var "y"))) (Var "z"))

λx.x y

["y"]

(λx.x y) z

["y","z"]

In [102]:
-- 단순무식한 substitition
-- dumbSubst x t s 는 s에서 x가 보이는대로 t로 바꿔친다
-- 이 함수의 문제점은 무엇일까?
dumbSubst :: Nm -> Tm -> Tm -> Tm
dumbSubst x t s@(Var y)   | x == y    = t
                          | otherwise = s
dumbSubst x t (Lam y s1)  = Lam y (dumbSubst x t s1)
dumbSubst x t (App s1 s2) = App (dumbSubst x t s1) (dumbSubst x t s2)

In [103]:
dispTm (Var "x")
dispTm $ dumbSubst "x" (App (Var "f") (Var "z")) (Var "x")

-- 자유(free)이름이 아닌 묶인(bound)이름까지 바꿔친다
dispTm ttTm
dispTm $ dumbSubst "x" (App (Var "f") (Var "z")) ttTm

x

f z

λx.λy.x

λx.λy.f z

In [104]:
-- 자유이름만 바꿔치지만 아직은 엉성한 substitition
-- 이 함수의 문제점은 무엇일까?
naiveSubst :: Nm -> Tm -> Tm -> Tm
naiveSubst x t s@(Var y)    | x == y    = t
                            | otherwise = s
naiveSubst x t s@(Lam y s1) | x == y    = s  -- 묶인이름은 바꿔치지마
                            | otherwise = Lam y (naiveSubst x t s1)
naiveSubst x t (App s1 s2)  = App (naiveSubst x t s1) (naiveSubst x t s2)

In [105]:
-- 묶인이름을 안바꿔치는 건 좋은데
dispTm ttTm
dispTm $ naiveSubst "x" (App (Var "f") (Var "z")) ttTm

-- 바꿔치고 보니 자유이름이 묶여서 갇힌다(capture) ㅠ.ㅠ
dispTm (Lam "z" (App (Var "x") (Var "z")))
dispTm $ naiveSubst "x" (App (Var "f") (Var "z")) (Lam "z" (App (Var "x") (Var "z")))

λx.λy.x

λx.λy.x

λz.x z

λz.f z z

In [106]:
-- 자유이름의 갇힘을 피하는 바꿔치기 capture avoding substutution
-- 우리가 원하는 이 바꿔치기 함수인 subst :: Nm -> Tm -> Tm -> Tm
-- 정의하고자 한다. 근데 이걸 정의하기 위해서는 필요한 개념이 있다.

In [107]:
isFresh :: Nm -> Tm -> Bool
isFresh x = notElem x . fv

In [108]:
dispTm $ Lam "x" (Var "x")
"x" `isFresh` Lam "x" (Var "x")

dispTm $ Lam "y" (Var "x")
"x" `isFresh` Lam "y" (Var "x")

λx.x

True

λy.x

False

In [109]:
{-
fresh x t 는 t에 대해 새로운 이름 즉 t의 자유이름들 중
어떤 것과도 겹치지 않으면서 앞부분이 x로 시작하는 이름을 구한다.
예를 들면 fresh "x" (Var "y") 의 경우라면 이미 "x"가 새롭기 때문에
그냥 "x"를 돌려줘도 무방하다. 하지만 fresh "x" (Var "x")의 경우는
"x"가 (Var "x")에 나타나는 자유이름이므로 "x1"이나 "x'" 혹은 "xx",
"xa", "xabc" 등 겹치지 않는 이름을 아무거나 하나 만들어내면 된다.
이것이 일반적으로 동작하도록 규칙을 나름대로 정해서 만들며 된다.
이런 fresh 함수를 작성해 보라. isFresh 함수를 활용하면 도움이 될 수 있다.
-}
fresh :: Nm -> Tm -> Nm
fresh x t = x -- 이것은 당연히 잘못된 dummy 구현이므로 고쳐야 한다

In [110]:
-- fresh 와 naiveSubst를 이용해 subst를 작성하라
subst :: Nm -> Tm -> Tm -> Tm
subst = naiveSubst -- 이것은 당연히 잘못된 dummy 구현이므로 고쳐야 한다

In [115]:
-- beta t 는 t로부터 가능한 모든 beta-줄이기(reduction) 한 걸음 결과를 리스트로 계산한다.
-- 줄일것(redex)이 없는 경우, 즉 t 가 beta-정규형(normal form)이면 빈 리스트를 돌려준다.
-- redexes 와 subst 를 사용하여 정의하라
beta :: Tm -> [Tm]
beta = map (tm'2tm . fmap betaTop) . redexes

betaTop (App (Lam x t) s) = subst x s t 
betaTop t = error $ "not a top level redex: " ++ ppTm t

In [116]:
mytm = App (App (Lam "u" (Var "u")) (App (Lam "x" (Var "x")) (Lam "y" (Var "y")))) (App (Lam "z" (Var "z")) (Var "w"))

dispTm mytm

(λu.u) ((λx.x) (λy.y)) ((λz.z) w)

In [117]:
map dispTm' $ redexes mytm

(λu.u) ((λx.x) (λy.y)) ((λz.z) w)

(λu.u) ( (λx.x) (λy.y) ) ((λz.z) w)

(λu.u) ((λx.x) (λy.y)) ( (λz.z) w )

In [118]:
map dispTm $ beta mytm

(λx.x) (λy.y) ((λz.z) w)

(λu.u) (λy.y) ((λz.z) w)

(λu.u) ((λx.x) (λy.y)) w

In [119]:
import Data.Tree
-- http://hackage.haskell.org/package/containers/docs/Data-Tree.html
{-
이걸 이용해서
beta reduction tree 만들어보기
-}